In [1]:
import pickle
import os
import numpy as np
from scipy.spatial import distance
import math
from math import radians, cos, sin, asin, sqrt
import random
import torch
import torch.utils.data as torchdata
from torch.nn import init
import time
from torch import nn
from gensim.models import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#建類別辭典，dname切換資料
dname='nyc'
#dname='tky'
data=pd.read_csv('../data/'+dname+'_filter_data.csv')
catDic={}
group=data.groupby('categid')
for gid,i in group:
    group_i=i.groupby('venueid')
    catnum=len(group_i)
    for cid , j in i.iterrows():
        catDic[j['categid']]=[j['categid'],j['catname'],len(i),catnum]
        break
    

catNum=len(catDic)
userNum=len(data.groupby('userid'))
poiNum=len(data.groupby('venueid'))
print(len(catDic))
print(userNum)
print(poiNum)

208
1083
5130


In [10]:
#正規畫
def norm(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

In [13]:
#類別使用者圖
catuserGraph=np.zeros((userNum+1,catNum,catNum))
group=data.groupby('userid')
for gid,j in group:
    j=j.sort_values(by='timestamp')
    outId=-1
    inId=-1
    past=0
    now=0
    for cid,i in j.iterrows():
        userid=i['userid']
        outId=inId
        inId=i['categid']
        past=now
        now=i['timestamp']
        time_int=now-past
        if outId!=-1 and time_int<24*60*60:
            catuserGraph[userid][outId][inId]+=1

np.save('../list/'+dname+'_usercat_graph',catuserGraph)
print(catuserGraph.shape)

(1084, 208, 208)


In [14]:
#正規化和平均
catuserGraph=np.load('../list/'+dname+'_usercat_graph.npy')
print(catuserGraph.shape)
c1=0
un=len(catuserGraph)
cn=len(catuserGraph[0])
for u in range(un):
    for i in range(cn):
        row_sum=np.sum(catuserGraph[u][i])
        if row_sum>0:
            catuserGraph[u][i]/=row_sum
            c1+=1
print(c1)
np.save('../list/'+dname+'_usercat_Percent',catuserGraph)


catuserGraph=np.load('../list/'+dname+'_usercat_graph.npy')
print(catuserGraph.shape)
c1=0
for u in range(un):
    for i in range(cn):
        row_sum=np.sum(catuserGraph[u][i])
        if row_sum>0:
            catuserGraph[u][i]=norm(catuserGraph[u][i])
            c1+=1
print(c1)
np.save('../list/'+dname+'_usercat_Norm',catuserGraph)


                


(1084, 208, 208)
15354
(1084, 208, 208)
15354


In [15]:
#使用者時間類別圖
ctGraph=np.zeros((userNum+1,catNum+48,catNum+48))
group=data.groupby('userid')
for gid,j in group:
    for cid,i in j.iterrows():
        userid=i['userid']
        time48=i['hour_48']
        cat=i['categid']
        ctGraph[userid][time48][cat+48]+=1
        ctGraph[userid][cat+48][time48]+=1

np.save('../list/'+dname+'_utc_graph',ctGraph)
print(ctGraph.shape)

(1084, 256, 256)


In [16]:
ctGraph=np.load('../list/'+dname+'_utc_graph.npy')
print(ctGraph.shape)
c1=0
un=len(ctGraph)
cn=len(ctGraph[0])
for u in range(un):
    for i in range(cn):
        row_sum=np.sum(ctGraph[u][i])
        if row_sum>0:
            ctGraph[u][i]/=row_sum
            c1+=1
print(c1)
np.save('../list/'+dname+'_utc_Percent',ctGraph)


ctGraph=np.load('../list/'+dname+'_utc_graph.npy')
print(ctGraph.shape)
c1=0
for u in range(un):
    for i in range(cn):
        row_sum=np.sum(ctGraph[u][i])
        if row_sum>0:
            ctGraph[u][i]=norm(ctGraph[u][i])
            c1+=1
print(c1)
np.save('../list/'+dname+'_utc_Norm',ctGraph)

(1084, 256, 256)
51606
(1084, 256, 256)
51606
